In [ ]:
%load_ext autoreload 
%autoreload 2

%matplotlib inline

In [ ]:
from src.lights import change_lights
from src.main import generate_scenario, run_scenario
from src.logger import ExperimentLogger

from utils.common import do_test_steps, create_simulator, get_camera_matrix, place_agent

In [ ]:
DEBUG = False

In [ ]:
sim_settings_filename = 'sim_settings_replica_cad.json'

In [ ]:
logger = ExperimentLogger(sim_settings_filename, display=True)

sim_settings, light_settings = logger.get_settings()

In [ ]:
try:
    sim.close()
except NameError:
    pass

sim = create_simulator(sim_settings)

In [ ]:
start_point, navigatable_points = generate_scenario(sim, sim_settings)

In [ ]:
sim = place_agent(sim, sim_settings['default_agent'], start_point)

In [ ]:
assert all(sim.agents[sim_settings['default_agent']].get_state().position == start_point)

In [ ]:
camera_matrix = get_camera_matrix(sim.agents[sim_settings['default_agent']])
logger.save_camera_params(camera_matrix, printed=True)

In [ ]:
if DEBUG:
    sim, _ = change_lights(sim, sim_settings, light_settings, life_index=0)
    do_test_steps(sim, sim_settings, max_frames=5)

In [ ]:
run_scenario(sim, sim_settings, light_settings, navigatable_points, logger)

In [ ]:
# !rm generated.zip
# !zip -r generated.zip generated/